In [2]:
import pandas as pd
import numpy as np

In [4]:
#NODES FOR PNU
nodes_df = pd.read_csv("CDRHourly_Flows/towers_index.txt", header = None)

#to get only lat & long from nodes_df 
len(nodes_df[[0,1]])

points = nodes_df[[0,1]]
import scipy.spatial as spsp
tree = spsp.cKDTree(points)
list_nodesnoID = tree.query_ball_point([24.852, 46.723], 0.005)
#to add 1 so that index = cell ID. (see Data_structures doc)
list_nodes =[x+1 for x in list_nodesnoID]

nodes_df['ID'] = nodes_df.index + 1
nodes_df.head()

,0,1,2,ID
0,24.517,46.776,20404,1
1,24.517,46.890,16208,2
2,24.517,46.890,16208,3
3,24.524,46.710,18805,4
4,24.525,46.648,11072,5


In [7]:
#DEFINING THE FUNCTION THAT PRINTS GEOJSONS

import json
def df_to_geojson(df, properties, lat='latitude', lon='longitude'):
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in df.iterrows():
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}
        feature['geometry']['coordinates'] = [row[2],row[1]]
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson

In [10]:
#6:00

df_06=pd.read_csv("CDRHourly_Flows/Rn_OD_6-7.txt", header=None, 
               names=['oid', 'did', 'close_rid_origin', 'close_rid_dest', 'flow'])

df_PNO = df_06[(df_06.isin({'oid':list_nodes}).any(1))]
df_PNO = df_PNO.drop('close_rid_origin',1)
df_PNO = df_PNO.drop('close_rid_dest',1)

#Join hourly flow info with specified nodes to get origins from PNU 
# df_PNO is flows from PNU i.e. PNU is the origin of flows  
df_PNO_xy =pd.merge(df_PNO, nodes_df[[0,1,'ID']], left_on = 'oid', right_on = 'ID', how = 'inner')
df_PNO_xy.rename(columns={0: 'lat_origin', 1: 'long_origin'}, inplace=True)
df_PNO_xy2 =pd.merge(df_PNO_xy, nodes_df[[0,1,'ID']], left_on = 'did', right_on = 'ID', how = 'inner')
df_PNO_xy2.rename(columns={0: 'lat_dest', 1: 'long_dest'}, inplace=True)
#drop the 'ID' columns to get a clean dataframe 
df_PNO_xy2 = df_PNO_xy2.drop('ID_x',1)
df_PNO_xy2 = df_PNO_xy2.drop('ID_y',1)

#creating a new dataframe that only has the did and flow

df_PNO_xy_active = df_PNO_xy2.drop([u'oid', u'lat_origin', u'long_origin'], 1)
df_PNO_xy_active2=(df_PNO_xy_active.groupby(['did','lat_dest','long_dest']).sum().reset_index())
df_PNO_xy_active2['hour']="6:00"

#PRINTING THE FILES

cols = ['did', 'lat_dest', 'long_dest', 'flow', 'hour']
geojson = df_to_geojson(df_PNO_xy_active2, cols)

output_filename = 'PNOhotspot_0607.geojson'
with open(output_filename, 'wb') as output_file:
    output_file.write('var dataset = ')
    json.dump(geojson, output_file, indent=2) 
    

df_PNO_xy_active2.to_csv('PNOhotspot_0607.csv')

In [11]:
#7:00

df_07=pd.read_csv("CDRHourly_Flows/Rn_OD_7-8.txt", header=None, 
               names=['oid', 'did', 'close_rid_origin', 'close_rid_dest', 'flow'])

df_PNO = df_07[(df_07.isin({'oid':list_nodes}).any(1))]
df_PNO = df_PNO.drop('close_rid_origin',1)
df_PNO = df_PNO.drop('close_rid_dest',1)

#Join hourly flow info with specified nodes to get origins from PNU 
# df_PNO is flows from PNU i.e. PNU is the origin of flows  
df_PNO_xy =pd.merge(df_PNO, nodes_df[[0,1,'ID']], left_on = 'oid', right_on = 'ID', how = 'inner')
df_PNO_xy.rename(columns={0: 'lat_origin', 1: 'long_origin'}, inplace=True)
df_PNO_xy2 =pd.merge(df_PNO_xy, nodes_df[[0,1,'ID']], left_on = 'did', right_on = 'ID', how = 'inner')
df_PNO_xy2.rename(columns={0: 'lat_dest', 1: 'long_dest'}, inplace=True)
#drop the 'ID' columns to get a clean dataframe 
df_PNO_xy2 = df_PNO_xy2.drop('ID_x',1)
df_PNO_xy2 = df_PNO_xy2.drop('ID_y',1)

#creating a new dataframe that only has the did and flow

df_PNO_xy_active = df_PNO_xy2.drop([u'oid', u'lat_origin', u'long_origin'], 1)
df_PNO_xy_active2=(df_PNO_xy_active.groupby(['did','lat_dest','long_dest']).sum().reset_index())
df_PNO_xy_active2['hour']="7:00"

#PRINTING THE FILES

cols = ['did', 'lat_dest', 'long_dest', 'flow', 'hour']
geojson = df_to_geojson(df_PNO_xy_active2, cols)

output_filename = 'PNOhotspot_0708.geojson'
with open(output_filename, 'wb') as output_file:
    output_file.write('var dataset = ')
    json.dump(geojson, output_file, indent=2) 
    

df_PNO_xy_active2.to_csv('PNOhotspot_0708.csv')

In [12]:
#8:00

df_08=pd.read_csv("CDRHourly_Flows/Rn_OD_8-9.txt", header=None, 
               names=['oid', 'did', 'close_rid_origin', 'close_rid_dest', 'flow'])

df_PNO = df_08[(df_08.isin({'oid':list_nodes}).any(1))]
df_PNO = df_PNO.drop('close_rid_origin',1)
df_PNO = df_PNO.drop('close_rid_dest',1)

#Join hourly flow info with specified nodes to get origins from PNU 
# df_PNO is flows from PNU i.e. PNU is the origin of flows  
df_PNO_xy =pd.merge(df_PNO, nodes_df[[0,1,'ID']], left_on = 'oid', right_on = 'ID', how = 'inner')
df_PNO_xy.rename(columns={0: 'lat_origin', 1: 'long_origin'}, inplace=True)
df_PNO_xy2 =pd.merge(df_PNO_xy, nodes_df[[0,1,'ID']], left_on = 'did', right_on = 'ID', how = 'inner')
df_PNO_xy2.rename(columns={0: 'lat_dest', 1: 'long_dest'}, inplace=True)
#drop the 'ID' columns to get a clean dataframe 
df_PNO_xy2 = df_PNO_xy2.drop('ID_x',1)
df_PNO_xy2 = df_PNO_xy2.drop('ID_y',1)

#creating a new dataframe that only has the did and flow

df_PNO_xy_active = df_PNO_xy2.drop([u'oid', u'lat_origin', u'long_origin'], 1)
df_PNO_xy_active2=(df_PNO_xy_active.groupby(['did','lat_dest','long_dest']).sum().reset_index())
df_PNO_xy_active2['hour']="8:00"

#PRINTING THE FILES

cols = ['did', 'lat_dest', 'long_dest', 'flow', 'hour']
geojson = df_to_geojson(df_PNO_xy_active2, cols)

output_filename = 'PNOhotspot_0809.geojson'
with open(output_filename, 'wb') as output_file:
    output_file.write('var dataset = ')
    json.dump(geojson, output_file, indent=2) 
    

df_PNO_xy_active2.to_csv('PNOhotspot_0809.csv')

In [9]:
#9:00

df_09=pd.read_csv("CDRHourly_Flows/Rn_OD_9-10.txt", header=None, 
               names=['oid', 'did', 'close_rid_origin', 'close_rid_dest', 'flow'])

df_PNO = df_09[(df_09.isin({'oid':list_nodes}).any(1))]
df_PNO = df_PNO.drop('close_rid_origin',1)
df_PNO = df_PNO.drop('close_rid_dest',1)

#Join hourly flow info with specified nodes to get origins from PNU 
# df_PNO is flows from PNU i.e. PNU is the origin of flows  
df_PNO_xy =pd.merge(df_PNO, nodes_df[[0,1,'ID']], left_on = 'oid', right_on = 'ID', how = 'inner')
df_PNO_xy.rename(columns={0: 'lat_origin', 1: 'long_origin'}, inplace=True)
df_PNO_xy2 =pd.merge(df_PNO_xy, nodes_df[[0,1,'ID']], left_on = 'did', right_on = 'ID', how = 'inner')
df_PNO_xy2.rename(columns={0: 'lat_dest', 1: 'long_dest'}, inplace=True)
#drop the 'ID' columns to get a clean dataframe 
df_PNO_xy2 = df_PNO_xy2.drop('ID_x',1)
df_PNO_xy2 = df_PNO_xy2.drop('ID_y',1)

#creating a new dataframe that only has the did and flow

df_PNO_xy_active = df_PNO_xy2.drop([u'oid', u'lat_origin', u'long_origin'], 1)
df_PNO_xy_active2=(df_PNO_xy_active.groupby(['did','lat_dest','long_dest']).sum().reset_index())
df_PNO_xy_active2['hour']="9:00"

#PRINTING THE FILES

cols = ['did', 'lat_dest', 'long_dest', 'flow', 'hour']
geojson = df_to_geojson(df_PNO_xy_active2, cols)

output_filename = 'PNOhotspot_0910.geojson'
with open(output_filename, 'wb') as output_file:
    output_file.write('var dataset = ')
    json.dump(geojson, output_file, indent=2) 
    

df_PNO_xy_active2.to_csv('PNOhotspot_0910.csv')

In [13]:
#10:00

df_10=pd.read_csv("CDRHourly_Flows/Rn_OD_10-11.txt", header=None, 
               names=['oid', 'did', 'close_rid_origin', 'close_rid_dest', 'flow'])

df_PNO = df_10[(df_10.isin({'oid':list_nodes}).any(1))]
df_PNO = df_PNO.drop('close_rid_origin',1)
df_PNO = df_PNO.drop('close_rid_dest',1)

#Join hourly flow info with specified nodes to get origins from PNU 
# df_PNO is flows from PNU i.e. PNU is the origin of flows  
df_PNO_xy =pd.merge(df_PNO, nodes_df[[0,1,'ID']], left_on = 'oid', right_on = 'ID', how = 'inner')
df_PNO_xy.rename(columns={0: 'lat_origin', 1: 'long_origin'}, inplace=True)
df_PNO_xy2 =pd.merge(df_PNO_xy, nodes_df[[0,1,'ID']], left_on = 'did', right_on = 'ID', how = 'inner')
df_PNO_xy2.rename(columns={0: 'lat_dest', 1: 'long_dest'}, inplace=True)
#drop the 'ID' columns to get a clean dataframe 
df_PNO_xy2 = df_PNO_xy2.drop('ID_x',1)
df_PNO_xy2 = df_PNO_xy2.drop('ID_y',1)

#creating a new dataframe that only has the did and flow

df_PNO_xy_active = df_PNO_xy2.drop([u'oid', u'lat_origin', u'long_origin'], 1)
df_PNO_xy_active2=(df_PNO_xy_active.groupby(['did','lat_dest','long_dest']).sum().reset_index())
df_PNO_xy_active2['hour']="10:00"

#PRINTING THE FILES

cols = ['did', 'lat_dest', 'long_dest', 'flow', 'hour']
geojson = df_to_geojson(df_PNO_xy_active2, cols)

output_filename = 'PNOhotspot_1011.geojson'
with open(output_filename, 'wb') as output_file:
    output_file.write('var dataset = ')
    json.dump(geojson, output_file, indent=2) 
    

df_PNO_xy_active2.to_csv('PNOhotspot_1011.csv')

In [14]:
#11:00

df_11=pd.read_csv("CDRHourly_Flows/Rn_OD_11-12.txt", header=None, 
               names=['oid', 'did', 'close_rid_origin', 'close_rid_dest', 'flow'])

df_PNO = df_11[(df_11.isin({'oid':list_nodes}).any(1))]
df_PNO = df_PNO.drop('close_rid_origin',1)
df_PNO = df_PNO.drop('close_rid_dest',1)

#Join hourly flow info with specified nodes to get origins from PNU 
# df_PNO is flows from PNU i.e. PNU is the origin of flows  
df_PNO_xy =pd.merge(df_PNO, nodes_df[[0,1,'ID']], left_on = 'oid', right_on = 'ID', how = 'inner')
df_PNO_xy.rename(columns={0: 'lat_origin', 1: 'long_origin'}, inplace=True)
df_PNO_xy2 =pd.merge(df_PNO_xy, nodes_df[[0,1,'ID']], left_on = 'did', right_on = 'ID', how = 'inner')
df_PNO_xy2.rename(columns={0: 'lat_dest', 1: 'long_dest'}, inplace=True)
#drop the 'ID' columns to get a clean dataframe 
df_PNO_xy2 = df_PNO_xy2.drop('ID_x',1)
df_PNO_xy2 = df_PNO_xy2.drop('ID_y',1)

#creating a new dataframe that only has the did and flow

df_PNO_xy_active = df_PNO_xy2.drop([u'oid', u'lat_origin', u'long_origin'], 1)
df_PNO_xy_active2=(df_PNO_xy_active.groupby(['did','lat_dest','long_dest']).sum().reset_index())
df_PNO_xy_active2['hour']="11:00"

#PRINTING THE FILES

cols = ['did', 'lat_dest', 'long_dest', 'flow', 'hour']
geojson = df_to_geojson(df_PNO_xy_active2, cols)

output_filename = 'PNOhotspot_1112.geojson'
with open(output_filename, 'wb') as output_file:
    output_file.write('var dataset = ')
    json.dump(geojson, output_file, indent=2) 
    

df_PNO_xy_active2.to_csv('PNOhotspot_1112.csv')

In [16]:
#12:00

df_12=pd.read_csv("CDRHourly_Flows/Rn_OD_12-13.txt", header=None, 
               names=['oid', 'did', 'close_rid_origin', 'close_rid_dest', 'flow'])

df_PNO = df_12[(df_12.isin({'oid':list_nodes}).any(1))]
df_PNO = df_PNO.drop('close_rid_origin',1)
df_PNO = df_PNO.drop('close_rid_dest',1)

#Join hourly flow info with specified nodes to get origins from PNU 
# df_PNO is flows from PNU i.e. PNU is the origin of flows  
df_PNO_xy =pd.merge(df_PNO, nodes_df[[0,1,'ID']], left_on = 'oid', right_on = 'ID', how = 'inner')
df_PNO_xy.rename(columns={0: 'lat_origin', 1: 'long_origin'}, inplace=True)
df_PNO_xy2 =pd.merge(df_PNO_xy, nodes_df[[0,1,'ID']], left_on = 'did', right_on = 'ID', how = 'inner')
df_PNO_xy2.rename(columns={0: 'lat_dest', 1: 'long_dest'}, inplace=True)
#drop the 'ID' columns to get a clean dataframe 
df_PNO_xy2 = df_PNO_xy2.drop('ID_x',1)
df_PNO_xy2 = df_PNO_xy2.drop('ID_y',1)

#creating a new dataframe that only has the did and flow

df_PNO_xy_active = df_PNO_xy2.drop([u'oid', u'lat_origin', u'long_origin'], 1)
df_PNO_xy_active2=(df_PNO_xy_active.groupby(['did','lat_dest','long_dest']).sum().reset_index())
df_PNO_xy_active2['hour']="12:00"

#PRINTING THE FILES

cols = ['did', 'lat_dest', 'long_dest', 'flow', 'hour']
geojson = df_to_geojson(df_PNO_xy_active2, cols)

output_filename = 'PNOhotspot_1213.geojson'
with open(output_filename, 'wb') as output_file:
    output_file.write('var dataset = ')
    json.dump(geojson, output_file, indent=2) 
    

df_PNO_xy_active2.to_csv('PNOhotspot_1213.csv')

In [17]:
#13:00

df_13=pd.read_csv("CDRHourly_Flows/Rn_OD_13-14.txt", header=None, 
               names=['oid', 'did', 'close_rid_origin', 'close_rid_dest', 'flow'])

df_PNO = df_13[(df_13.isin({'oid':list_nodes}).any(1))]
df_PNO = df_PNO.drop('close_rid_origin',1)
df_PNO = df_PNO.drop('close_rid_dest',1)

#Join hourly flow info with specified nodes to get origins from PNU 
# df_PNO is flows from PNU i.e. PNU is the origin of flows  
df_PNO_xy =pd.merge(df_PNO, nodes_df[[0,1,'ID']], left_on = 'oid', right_on = 'ID', how = 'inner')
df_PNO_xy.rename(columns={0: 'lat_origin', 1: 'long_origin'}, inplace=True)
df_PNO_xy2 =pd.merge(df_PNO_xy, nodes_df[[0,1,'ID']], left_on = 'did', right_on = 'ID', how = 'inner')
df_PNO_xy2.rename(columns={0: 'lat_dest', 1: 'long_dest'}, inplace=True)
#drop the 'ID' columns to get a clean dataframe 
df_PNO_xy2 = df_PNO_xy2.drop('ID_x',1)
df_PNO_xy2 = df_PNO_xy2.drop('ID_y',1)

#creating a new dataframe that only has the did and flow

df_PNO_xy_active = df_PNO_xy2.drop([u'oid', u'lat_origin', u'long_origin'], 1)
df_PNO_xy_active2=(df_PNO_xy_active.groupby(['did','lat_dest','long_dest']).sum().reset_index())
df_PNO_xy_active2['hour']="13:00"

#PRINTING THE FILES

cols = ['did', 'lat_dest', 'long_dest', 'flow', 'hour']
geojson = df_to_geojson(df_PNO_xy_active2, cols)

output_filename = 'PNOhotspot_1314.geojson'
with open(output_filename, 'wb') as output_file:
    output_file.write('var dataset = ')
    json.dump(geojson, output_file, indent=2) 
    

df_PNO_xy_active2.to_csv('PNOhotspot_1314.csv')

In [18]:
#14:00

df_14=pd.read_csv("CDRHourly_Flows/Rn_OD_14-15.txt", header=None, 
               names=['oid', 'did', 'close_rid_origin', 'close_rid_dest', 'flow'])

df_PNO = df_14[(df_14.isin({'oid':list_nodes}).any(1))]
df_PNO = df_PNO.drop('close_rid_origin',1)
df_PNO = df_PNO.drop('close_rid_dest',1)

#Join hourly flow info with specified nodes to get origins from PNU 
# df_PNO is flows from PNU i.e. PNU is the origin of flows  
df_PNO_xy =pd.merge(df_PNO, nodes_df[[0,1,'ID']], left_on = 'oid', right_on = 'ID', how = 'inner')
df_PNO_xy.rename(columns={0: 'lat_origin', 1: 'long_origin'}, inplace=True)
df_PNO_xy2 =pd.merge(df_PNO_xy, nodes_df[[0,1,'ID']], left_on = 'did', right_on = 'ID', how = 'inner')
df_PNO_xy2.rename(columns={0: 'lat_dest', 1: 'long_dest'}, inplace=True)
#drop the 'ID' columns to get a clean dataframe 
df_PNO_xy2 = df_PNO_xy2.drop('ID_x',1)
df_PNO_xy2 = df_PNO_xy2.drop('ID_y',1)

#creating a new dataframe that only has the did and flow

df_PNO_xy_active = df_PNO_xy2.drop([u'oid', u'lat_origin', u'long_origin'], 1)
df_PNO_xy_active2=(df_PNO_xy_active.groupby(['did','lat_dest','long_dest']).sum().reset_index())
df_PNO_xy_active2['hour']="14:00"

#PRINTING THE FILES

cols = ['did', 'lat_dest', 'long_dest', 'flow', 'hour']
geojson = df_to_geojson(df_PNO_xy_active2, cols)

output_filename = 'PNOhotspot_1415.geojson'
with open(output_filename, 'wb') as output_file:
    output_file.write('var dataset = ')
    json.dump(geojson, output_file, indent=2) 
    

df_PNO_xy_active2.to_csv('PNOhotspot_1415.csv')

In [19]:
#15:00

df_15=pd.read_csv("CDRHourly_Flows/Rn_OD_15-16.txt", header=None, 
               names=['oid', 'did', 'close_rid_origin', 'close_rid_dest', 'flow'])

df_PNO = df_15[(df_15.isin({'oid':list_nodes}).any(1))]
df_PNO = df_PNO.drop('close_rid_origin',1)
df_PNO = df_PNO.drop('close_rid_dest',1)

#Join hourly flow info with specified nodes to get origins from PNU 
# df_PNO is flows from PNU i.e. PNU is the origin of flows  
df_PNO_xy =pd.merge(df_PNO, nodes_df[[0,1,'ID']], left_on = 'oid', right_on = 'ID', how = 'inner')
df_PNO_xy.rename(columns={0: 'lat_origin', 1: 'long_origin'}, inplace=True)
df_PNO_xy2 =pd.merge(df_PNO_xy, nodes_df[[0,1,'ID']], left_on = 'did', right_on = 'ID', how = 'inner')
df_PNO_xy2.rename(columns={0: 'lat_dest', 1: 'long_dest'}, inplace=True)
#drop the 'ID' columns to get a clean dataframe 
df_PNO_xy2 = df_PNO_xy2.drop('ID_x',1)
df_PNO_xy2 = df_PNO_xy2.drop('ID_y',1)

#creating a new dataframe that only has the did and flow

df_PNO_xy_active = df_PNO_xy2.drop([u'oid', u'lat_origin', u'long_origin'], 1)
df_PNO_xy_active2=(df_PNO_xy_active.groupby(['did','lat_dest','long_dest']).sum().reset_index())
df_PNO_xy_active2['hour']="15:00"

#PRINTING THE FILES

cols = ['did', 'lat_dest', 'long_dest', 'flow', 'hour']
geojson = df_to_geojson(df_PNO_xy_active2, cols)

output_filename = 'PNOhotspot_1516.geojson'
with open(output_filename, 'wb') as output_file:
    output_file.write('var dataset = ')
    json.dump(geojson, output_file, indent=2) 
    

df_PNO_xy_active2.to_csv('PNOhotspot_1516.csv')

In [20]:
#16:00

df_16=pd.read_csv("CDRHourly_Flows/Rn_OD_16-17.txt", header=None, 
               names=['oid', 'did', 'close_rid_origin', 'close_rid_dest', 'flow'])

df_PNO = df_16[(df_16.isin({'oid':list_nodes}).any(1))]
df_PNO = df_PNO.drop('close_rid_origin',1)
df_PNO = df_PNO.drop('close_rid_dest',1)

#Join hourly flow info with specified nodes to get origins from PNU 
# df_PNO is flows from PNU i.e. PNU is the origin of flows  
df_PNO_xy =pd.merge(df_PNO, nodes_df[[0,1,'ID']], left_on = 'oid', right_on = 'ID', how = 'inner')
df_PNO_xy.rename(columns={0: 'lat_origin', 1: 'long_origin'}, inplace=True)
df_PNO_xy2 =pd.merge(df_PNO_xy, nodes_df[[0,1,'ID']], left_on = 'did', right_on = 'ID', how = 'inner')
df_PNO_xy2.rename(columns={0: 'lat_dest', 1: 'long_dest'}, inplace=True)
#drop the 'ID' columns to get a clean dataframe 
df_PNO_xy2 = df_PNO_xy2.drop('ID_x',1)
df_PNO_xy2 = df_PNO_xy2.drop('ID_y',1)

#creating a new dataframe that only has the did and flow

df_PNO_xy_active = df_PNO_xy2.drop([u'oid', u'lat_origin', u'long_origin'], 1)
df_PNO_xy_active2=(df_PNO_xy_active.groupby(['did','lat_dest','long_dest']).sum().reset_index())
df_PNO_xy_active2['hour']="16:00"

#PRINTING THE FILES

cols = ['did', 'lat_dest', 'long_dest', 'flow', 'hour']
geojson = df_to_geojson(df_PNO_xy_active2, cols)

output_filename = 'PNOhotspot_1617.geojson'
with open(output_filename, 'wb') as output_file:
    output_file.write('var dataset = ')
    json.dump(geojson, output_file, indent=2) 
    

df_PNO_xy_active2.to_csv('PNOhotspot_1617.csv')

In [21]:
#17:00

df_17=pd.read_csv("CDRHourly_Flows/Rn_OD_17-18.txt", header=None, 
               names=['oid', 'did', 'close_rid_origin', 'close_rid_dest', 'flow'])

df_PNO = df_17[(df_17.isin({'oid':list_nodes}).any(1))]
df_PNO = df_PNO.drop('close_rid_origin',1)
df_PNO = df_PNO.drop('close_rid_dest',1)

#Join hourly flow info with specified nodes to get origins from PNU 
# df_PNO is flows from PNU i.e. PNU is the origin of flows  
df_PNO_xy =pd.merge(df_PNO, nodes_df[[0,1,'ID']], left_on = 'oid', right_on = 'ID', how = 'inner')
df_PNO_xy.rename(columns={0: 'lat_origin', 1: 'long_origin'}, inplace=True)
df_PNO_xy2 =pd.merge(df_PNO_xy, nodes_df[[0,1,'ID']], left_on = 'did', right_on = 'ID', how = 'inner')
df_PNO_xy2.rename(columns={0: 'lat_dest', 1: 'long_dest'}, inplace=True)
#drop the 'ID' columns to get a clean dataframe 
df_PNO_xy2 = df_PNO_xy2.drop('ID_x',1)
df_PNO_xy2 = df_PNO_xy2.drop('ID_y',1)

#creating a new dataframe that only has the did and flow

df_PNO_xy_active = df_PNO_xy2.drop([u'oid', u'lat_origin', u'long_origin'], 1)
df_PNO_xy_active2=(df_PNO_xy_active.groupby(['did','lat_dest','long_dest']).sum().reset_index())
df_PNO_xy_active2['hour']="17:00"

#PRINTING THE FILES

cols = ['did', 'lat_dest', 'long_dest', 'flow', 'hour']
geojson = df_to_geojson(df_PNO_xy_active2, cols)

output_filename = 'PNOhotspot_1718.geojson'
with open(output_filename, 'wb') as output_file:
    output_file.write('var dataset = ')
    json.dump(geojson, output_file, indent=2) 
    

df_PNO_xy_active2.to_csv('PNOhotspot_1718.csv')

In [22]:
#18:00

df_18=pd.read_csv("CDRHourly_Flows/Rn_OD_18-19.txt", header=None, 
               names=['oid', 'did', 'close_rid_origin', 'close_rid_dest', 'flow'])

df_PNO = df_18[(df_18.isin({'oid':list_nodes}).any(1))]
df_PNO = df_PNO.drop('close_rid_origin',1)
df_PNO = df_PNO.drop('close_rid_dest',1)

#Join hourly flow info with specified nodes to get origins from PNU 
# df_PNO is flows from PNU i.e. PNU is the origin of flows  
df_PNO_xy =pd.merge(df_PNO, nodes_df[[0,1,'ID']], left_on = 'oid', right_on = 'ID', how = 'inner')
df_PNO_xy.rename(columns={0: 'lat_origin', 1: 'long_origin'}, inplace=True)
df_PNO_xy2 =pd.merge(df_PNO_xy, nodes_df[[0,1,'ID']], left_on = 'did', right_on = 'ID', how = 'inner')
df_PNO_xy2.rename(columns={0: 'lat_dest', 1: 'long_dest'}, inplace=True)
#drop the 'ID' columns to get a clean dataframe 
df_PNO_xy2 = df_PNO_xy2.drop('ID_x',1)
df_PNO_xy2 = df_PNO_xy2.drop('ID_y',1)

#creating a new dataframe that only has the did and flow

df_PNO_xy_active = df_PNO_xy2.drop([u'oid', u'lat_origin', u'long_origin'], 1)
df_PNO_xy_active2=(df_PNO_xy_active.groupby(['did','lat_dest','long_dest']).sum().reset_index())
df_PNO_xy_active2['hour']="18:00"

#PRINTING THE FILES

cols = ['did', 'lat_dest', 'long_dest', 'flow', 'hour']
geojson = df_to_geojson(df_PNO_xy_active2, cols)

output_filename = 'PNOhotspot_1819.geojson'
with open(output_filename, 'wb') as output_file:
    output_file.write('var dataset = ')
    json.dump(geojson, output_file, indent=2) 
    

df_PNO_xy_active2.to_csv('PNOhotspot_1819.csv')

In [23]:
#19:00

df_19=pd.read_csv("CDRHourly_Flows/Rn_OD_19-20.txt", header=None, 
               names=['oid', 'did', 'close_rid_origin', 'close_rid_dest', 'flow'])

df_PNO = df_19[(df_19.isin({'oid':list_nodes}).any(1))]
df_PNO = df_PNO.drop('close_rid_origin',1)
df_PNO = df_PNO.drop('close_rid_dest',1)

#Join hourly flow info with specified nodes to get origins from PNU 
# df_PNO is flows from PNU i.e. PNU is the origin of flows  
df_PNO_xy =pd.merge(df_PNO, nodes_df[[0,1,'ID']], left_on = 'oid', right_on = 'ID', how = 'inner')
df_PNO_xy.rename(columns={0: 'lat_origin', 1: 'long_origin'}, inplace=True)
df_PNO_xy2 =pd.merge(df_PNO_xy, nodes_df[[0,1,'ID']], left_on = 'did', right_on = 'ID', how = 'inner')
df_PNO_xy2.rename(columns={0: 'lat_dest', 1: 'long_dest'}, inplace=True)
#drop the 'ID' columns to get a clean dataframe 
df_PNO_xy2 = df_PNO_xy2.drop('ID_x',1)
df_PNO_xy2 = df_PNO_xy2.drop('ID_y',1)

#creating a new dataframe that only has the did and flow

df_PNO_xy_active = df_PNO_xy2.drop([u'oid', u'lat_origin', u'long_origin'], 1)
df_PNO_xy_active2=(df_PNO_xy_active.groupby(['did','lat_dest','long_dest']).sum().reset_index())
df_PNO_xy_active2['hour']="19:00"

#PRINTING THE FILES

cols = ['did', 'lat_dest', 'long_dest', 'flow', 'hour']
geojson = df_to_geojson(df_PNO_xy_active2, cols)

output_filename = 'PNOhotspot_1920.geojson'
with open(output_filename, 'wb') as output_file:
    output_file.write('var dataset = ')
    json.dump(geojson, output_file, indent=2) 
    

df_PNO_xy_active2.to_csv('PNOhotspot_1920.csv')

In [26]:
#20:00

df_20=pd.read_csv("CDRHourly_Flows/Rn_OD_20-21.txt", header=None, 
               names=['oid', 'did', 'close_rid_origin', 'close_rid_dest', 'flow'])

df_PNO = df_20[(df_20.isin({'oid':list_nodes}).any(1))]
df_PNO = df_PNO.drop('close_rid_origin',1)
df_PNO = df_PNO.drop('close_rid_dest',1)

#Join hourly flow info with specified nodes to get origins from PNU 
# df_PNO is flows from PNU i.e. PNU is the origin of flows  
df_PNO_xy =pd.merge(df_PNO, nodes_df[[0,1,'ID']], left_on = 'oid', right_on = 'ID', how = 'inner')
df_PNO_xy.rename(columns={0: 'lat_origin', 1: 'long_origin'}, inplace=True)
df_PNO_xy2 =pd.merge(df_PNO_xy, nodes_df[[0,1,'ID']], left_on = 'did', right_on = 'ID', how = 'inner')
df_PNO_xy2.rename(columns={0: 'lat_dest', 1: 'long_dest'}, inplace=True)
#drop the 'ID' columns to get a clean dataframe 
df_PNO_xy2 = df_PNO_xy2.drop('ID_x',1)
df_PNO_xy2 = df_PNO_xy2.drop('ID_y',1)

#creating a new dataframe that only has the did and flow

df_PNO_xy_active = df_PNO_xy2.drop([u'oid', u'lat_origin', u'long_origin'], 1)
df_PNO_xy_active2=(df_PNO_xy_active.groupby(['did','lat_dest','long_dest']).sum().reset_index())
df_PNO_xy_active2['hour']="20:00"

#PRINTING THE FILES

cols = ['did', 'lat_dest', 'long_dest', 'flow', 'hour']
geojson = df_to_geojson(df_PNO_xy_active2, cols)

output_filename = 'PNOhotspot_2021.geojson'
with open(output_filename, 'wb') as output_file:
    output_file.write('var dataset = ')
    json.dump(geojson, output_file, indent=2) 
    

df_PNO_xy_active2.to_csv('PNOhotspot_2021.csv')

In [27]:
#21:00

df_21=pd.read_csv("CDRHourly_Flows/Rn_OD_21-22.txt", header=None, 
               names=['oid', 'did', 'close_rid_origin', 'close_rid_dest', 'flow'])

df_PNO = df_21[(df_21.isin({'oid':list_nodes}).any(1))]
df_PNO = df_PNO.drop('close_rid_origin',1)
df_PNO = df_PNO.drop('close_rid_dest',1)

#Join hourly flow info with specified nodes to get origins from PNU 
# df_PNO is flows from PNU i.e. PNU is the origin of flows  
df_PNO_xy =pd.merge(df_PNO, nodes_df[[0,1,'ID']], left_on = 'oid', right_on = 'ID', how = 'inner')
df_PNO_xy.rename(columns={0: 'lat_origin', 1: 'long_origin'}, inplace=True)
df_PNO_xy2 =pd.merge(df_PNO_xy, nodes_df[[0,1,'ID']], left_on = 'did', right_on = 'ID', how = 'inner')
df_PNO_xy2.rename(columns={0: 'lat_dest', 1: 'long_dest'}, inplace=True)
#drop the 'ID' columns to get a clean dataframe 
df_PNO_xy2 = df_PNO_xy2.drop('ID_x',1)
df_PNO_xy2 = df_PNO_xy2.drop('ID_y',1)

#creating a new dataframe that only has the did and flow

df_PNO_xy_active = df_PNO_xy2.drop([u'oid', u'lat_origin', u'long_origin'], 1)
df_PNO_xy_active2=(df_PNO_xy_active.groupby(['did','lat_dest','long_dest']).sum().reset_index())
df_PNO_xy_active2['hour']="21:00"

#PRINTING THE FILES

cols = ['did', 'lat_dest', 'long_dest', 'flow', 'hour']
geojson = df_to_geojson(df_PNO_xy_active2, cols)

output_filename = 'PNOhotspot_2122.geojson'
with open(output_filename, 'wb') as output_file:
    output_file.write('var dataset = ')
    json.dump(geojson, output_file, indent=2) 
    

df_PNO_xy_active2.to_csv('PNOhotspot_2122.csv')

In [29]:
#22:00

df_22=pd.read_csv("CDRHourly_Flows/Rn_OD_22-23.txt", header=None, 
               names=['oid', 'did', 'close_rid_origin', 'close_rid_dest', 'flow'])

df_PNO = df_22[(df_22.isin({'oid':list_nodes}).any(1))]
df_PNO = df_PNO.drop('close_rid_origin',1)
df_PNO = df_PNO.drop('close_rid_dest',1)

#Join hourly flow info with specified nodes to get origins from PNU 
# df_PNO is flows from PNU i.e. PNU is the origin of flows  
df_PNO_xy =pd.merge(df_PNO, nodes_df[[0,1,'ID']], left_on = 'oid', right_on = 'ID', how = 'inner')
df_PNO_xy.rename(columns={0: 'lat_origin', 1: 'long_origin'}, inplace=True)
df_PNO_xy2 =pd.merge(df_PNO_xy, nodes_df[[0,1,'ID']], left_on = 'did', right_on = 'ID', how = 'inner')
df_PNO_xy2.rename(columns={0: 'lat_dest', 1: 'long_dest'}, inplace=True)
#drop the 'ID' columns to get a clean dataframe 
df_PNO_xy2 = df_PNO_xy2.drop('ID_x',1)
df_PNO_xy2 = df_PNO_xy2.drop('ID_y',1)

#creating a new dataframe that only has the did and flow

df_PNO_xy_active = df_PNO_xy2.drop([u'oid', u'lat_origin', u'long_origin'], 1)
df_PNO_xy_active2=(df_PNO_xy_active.groupby(['did','lat_dest','long_dest']).sum().reset_index())
df_PNO_xy_active2['hour']="22:00"

#PRINTING THE FILES

cols = ['did', 'lat_dest', 'long_dest', 'flow', 'hour']
geojson = df_to_geojson(df_PNO_xy_active2, cols)

output_filename = 'PNOhotspot_2223.geojson'
with open(output_filename, 'wb') as output_file:
    output_file.write('var dataset = ')
    json.dump(geojson, output_file, indent=2) 
    

df_PNO_xy_active2.to_csv('PNOhotspot_2223.csv')

In [31]:
#23:00

df_23=pd.read_csv("CDRHourly_Flows/Rn_OD_23-24.txt", header=None, 
               names=['oid', 'did', 'close_rid_origin', 'close_rid_dest', 'flow'])

df_PNO = df_23[(df_23.isin({'oid':list_nodes}).any(1))]
df_PNO = df_PNO.drop('close_rid_origin',1)
df_PNO = df_PNO.drop('close_rid_dest',1)

#Join hourly flow info with specified nodes to get origins from PNU 
# df_PNO is flows from PNU i.e. PNU is the origin of flows  
df_PNO_xy =pd.merge(df_PNO, nodes_df[[0,1,'ID']], left_on = 'oid', right_on = 'ID', how = 'inner')
df_PNO_xy.rename(columns={0: 'lat_origin', 1: 'long_origin'}, inplace=True)
df_PNO_xy2 =pd.merge(df_PNO_xy, nodes_df[[0,1,'ID']], left_on = 'did', right_on = 'ID', how = 'inner')
df_PNO_xy2.rename(columns={0: 'lat_dest', 1: 'long_dest'}, inplace=True)
#drop the 'ID' columns to get a clean dataframe 
df_PNO_xy2 = df_PNO_xy2.drop('ID_x',1)
df_PNO_xy2 = df_PNO_xy2.drop('ID_y',1)

#creating a new dataframe that only has the did and flow

df_PNO_xy_active = df_PNO_xy2.drop([u'oid', u'lat_origin', u'long_origin'], 1)
df_PNO_xy_active2=(df_PNO_xy_active.groupby(['did','lat_dest','long_dest']).sum().reset_index())
df_PNO_xy_active2['hour']="23:00"

#PRINTING THE FILES

cols = ['did', 'lat_dest', 'long_dest', 'flow', 'hour']
geojson = df_to_geojson(df_PNO_xy_active2, cols)

output_filename = 'PNOhotspot_2324.geojson'
with open(output_filename, 'wb') as output_file:
    output_file.write('var dataset = ')
    json.dump(geojson, output_file, indent=2) 
    

df_PNO_xy_active2.to_csv('PNOhotspot_2324.csv')

In [32]:
#00:00

df_0=pd.read_csv("CDRHourly_Flows/Rn_OD_0-1.txt", header=None, 
               names=['oid', 'did', 'close_rid_origin', 'close_rid_dest', 'flow'])

df_PNO = df_0[(df_0.isin({'oid':list_nodes}).any(1))]
df_PNO = df_PNO.drop('close_rid_origin',1)
df_PNO = df_PNO.drop('close_rid_dest',1)

#Join hourly flow info with specified nodes to get origins from PNU 
# df_PNO is flows from PNU i.e. PNU is the origin of flows  
df_PNO_xy =pd.merge(df_PNO, nodes_df[[0,1,'ID']], left_on = 'oid', right_on = 'ID', how = 'inner')
df_PNO_xy.rename(columns={0: 'lat_origin', 1: 'long_origin'}, inplace=True)
df_PNO_xy2 =pd.merge(df_PNO_xy, nodes_df[[0,1,'ID']], left_on = 'did', right_on = 'ID', how = 'inner')
df_PNO_xy2.rename(columns={0: 'lat_dest', 1: 'long_dest'}, inplace=True)
#drop the 'ID' columns to get a clean dataframe 
df_PNO_xy2 = df_PNO_xy2.drop('ID_x',1)
df_PNO_xy2 = df_PNO_xy2.drop('ID_y',1)

#creating a new dataframe that only has the did and flow

df_PNO_xy_active = df_PNO_xy2.drop([u'oid', u'lat_origin', u'long_origin'], 1)
df_PNO_xy_active2=(df_PNO_xy_active.groupby(['did','lat_dest','long_dest']).sum().reset_index())
df_PNO_xy_active2['hour']="24:00"

#PRINTING THE FILES

cols = ['did', 'lat_dest', 'long_dest', 'flow', 'hour']
geojson = df_to_geojson(df_PNO_xy_active2, cols)

output_filename = 'PNOhotspot_0001.geojson'
with open(output_filename, 'wb') as output_file:
    output_file.write('var dataset = ')
    json.dump(geojson, output_file, indent=2) 
    

df_PNO_xy_active2.to_csv('PNOhotspot_0001.csv')